# Log data  from Hubway for 24 h

5 Oct 2015

In [1]:
from pylab import rcParams
%matplotlib inline
rcParams['figure.figsize'] = (8,6)

import contextlib
import urllib
from lxml import etree
import datetime
import pandas as pd
import time
import os

In [2]:
def getupdate():
    u = 'http://www.thehubway.com/data/stations/bikeStations.xml'
    # tweak of urlopen from hubway_repeatingquery, see http://stackoverflow.com/a/1522709
    with contextlib.closing(urllib.urlopen(u)) as x:
        data = etree.parse(x)
    stations = data.findall('station')
    everything = [[elt.text for elt in station.getchildren()] for station in stations]
    df = pd.DataFrame(everything, columns = [elt.tag for elt in data.find('station')]).convert_objects(convert_numeric=True)
    df.set_index('name', inplace=True)
    return df

In [3]:
def logdata(quantity, interval, numpts):
    df = getupdate()
    dflog = pd.DataFrame(df[quantity])
    dflog.columns = [datetime.datetime.today()]
    i = 1
    while i < numpts:
        time.sleep(interval)
        df = getupdate()
        dflog[datetime.datetime.today()] = df[quantity]
        i = i+1
    return dflog.transpose()

In [4]:
# let's let it run for a 24 hour at 1 Hz...
# except oops, ran for 24 h at 1/24 h!
bikelog = logdata('nbBikes', 60*24, 60)

In [5]:
bikelog.head()

name,Colleges of the Fenway,Tremont St. at Berkeley St.,Northeastern U / North Parking Lot,Cambridge St. at Joy St.,Fan Pier,Union Square - Brighton Ave. at Cambridge St.,Agganis Arena - 925 Comm Ave.,B.U. Central - 725 Comm. Ave.,Longwood Ave / Binney St,Ruggles Station / Columbus Ave.,...,Alewife Station at Russell Field,EF - North Point Park,Rindge Avenue - O'Neill Library,Harvard University Transportation Services - 175 North Harvard St,State Street at Channel Center,John F Fitzgerald - Surface Road at India Street,Ink Block,Wentworth Institute of Technology,Seaver Street at Humbolt Ave,West Broadway at D Street
2015-10-05 07:24:24.015,2,4,8,3,8,13,15,5,13,4,...,18,13,8,10,11,12,11,3,3,8
2015-10-05 07:48:24.096,2,0,8,0,2,13,15,6,14,2,...,18,13,6,10,10,13,9,3,2,8
2015-10-05 08:12:24.176,3,1,9,1,1,13,15,8,14,3,...,18,12,6,6,10,11,7,3,2,8
2015-10-05 08:36:24.285,5,0,10,3,5,12,17,8,11,1,...,20,14,6,3,13,15,7,3,2,8
2015-10-05 09:00:24.373,9,1,11,4,9,11,17,9,9,2,...,21,19,4,2,16,18,4,4,1,8


## Export logdata to CSV

In [6]:
bikelog.to_csv('bikelog_100515.csv')